In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff

#from absl import app
#import nest_asyncio
#nest_asyncio.apply()

#from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

import gower as gd

#import sys
#sys.path.append("/home/tester/Desktop/TF/federated/tensorflow_federated/examples/simple_fedavg")
#sys.path.append("/home/tester/Desktop/TF/gower/gower")
#import simple_fedavg_tff
#import gower_dist as gd


In [13]:
RUN_NAME = 'F2'
PRINT_SCR = True
BALANCE_DATA = False
SAMPLE_SIZE = 5000
TEST_SIZE = 475

In [14]:
#Load model
model = tf.keras.models.load_model('results/fl/' + RUN_NAME + '/model.h5')

In [15]:
df = pd.read_csv("../datasets/TON_IoT-Datasets/Train_Test_datasets/Train_Test_Network_dataset/Train_Test_Network.csv")
df.pop('type')
df.pop('ts')
df.head()

,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label
0,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,OTH,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
1,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,S0,...,0,0,0,-,-,-,-,-,-,0
2,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,OTH,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
3,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,OTH,...,0,0,0,-,-,-,-,-,-,0
4,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,SHR,...,0,0,0,-,-,-,bad_UDP_checksum,-,F,0


In [16]:
cat_indexs = [0, 1, 2, 3, 4, 5, 9, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 35, 36, 37, 38, 39, 40, 41]
num_indexs = [6, 7, 8, 10, 11, 12, 13, 14, 33, 34]
#bool_indexs = []


In [17]:
# Which cols are categorical
cat_index_bool = [False] * 42
for e in cat_indexs:
    cat_index_bool[e] = True

In [18]:
# # Percentage malware
perc = len(df.loc[df['label']==1])/len(df)
if PRINT_SCR:
    print(perc)

# Balance dataset
if BALANCE_DATA:
    num_anom = len(df.loc[df['label']==1])
    df_anom = df.loc[df['label']==1]
    df_balance = df.loc[df['label']==0]
    df_balance = df_balance.sample(num_anom, replace=False)
    df_concated = pd.concat([df_balance, df_anom])
    balanced_data = df_concated
    df = balanced_data


0.34930147513355586


In [19]:
test_data = df.sample(SAMPLE_SIZE, random_state=2)
test_labels = test_data.pop('label')

In [20]:
# Need train partition to compute test matrix
test_gower_mat = gd.gower_matrix_limit_cols(test_data,TEST_SIZE,cat_features=cat_index_bool)

In [21]:
def predict(model, data):
  reconstructions = model(data)
  return reconstructions

def save_stats(predictions, labels, print_scr=True):

  acc = "{:.4f}".format(accuracy_score(labels, predictions))
  prec = "{:.4f}".format(precision_score(labels, predictions, zero_division=0))
  rcl = "{:.4f}".format(recall_score(labels, predictions, zero_division=0))
  f1 = "{:.4f}".format(f1_score(labels, predictions, zero_division=0))
  roc = "{:.4f}".format(roc_auc_score(labels, predictions))

  if print_scr:
    print("Accuracy = " + acc)
    print("Precision = " + prec)
    print("Recall = " + rcl) #sensitivity
    print("F1 = " + f1)
    print("ROC_AUC = " + roc)

  return (acc + " " + prec + " " + rcl + " " + f1 + " " + roc)


In [22]:
#test = pd.DataFrame.from_dict(test_cl_dict[str(np.random.randint(0,NUM_CLIENTS-1))]).copy()
#labs = test.pop('label')
preds = predict(model, np.array(test_gower_mat))
a = save_stats(np.round(preds[:,1], decimals=0), test_labels.astype(int), PRINT_SCR)

Accuracy = 0.6476
Precision = 0.0000
Recall = 0.0000
F1 = 0.0000
ROC_AUC = 0.5000


In [23]:
#first_layer_weights = model.layers[0].get_weights()[0]
#print(first_layer_weights)

[[-0.0795357  -0.04372541 -0.11215445 ... -0.03973442 -0.06170009
  -0.02951626]
 [-0.01693022 -0.08426588 -0.08953209 ...  0.00368234  0.00227912
  -0.08559938]
 [ 0.00039241  0.09665311 -0.03616171 ... -0.10342579  0.01036007
   0.0136985 ]
 ...
 [ 0.01536194 -0.03581058  0.1203389  ... -0.01302702 -0.01485945
  -0.03442252]
 [ 0.0404333   0.09487855 -0.02929065 ... -0.06922565  0.03077221
  -0.04699507]
 [-0.05709641 -0.02988226  0.0511932  ... -0.07617711 -0.12072495
  -0.0644127 ]]
